# JSON COMPILER

In [ ]:
import json
import os
from collections import defaultdict

# === Configuration ===
json_directory = r"json"
json_files = [
    "employee_master", "emp_references", "emp_language", "emp_hobbies",
    "emp_family", "emp_experience", "emp_education", "emp_documents",
    "emp_curricular", "emp_contact", "emp_certification"
]

employee_data = defaultdict(lambda: defaultdict(list))

print("🔍 Starting file processing...\n")

for file_name in json_files:
    file_path = os.path.join(json_directory, file_name + ".json")
    if not os.path.exists(file_path):
        print(f"❌ Missing file: {file_path}")
        continue

    with open(file_path, "r", encoding="utf-8") as f:
        try:
            content = json.load(f)
        except json.JSONDecodeError as e:
            print(f"❌ JSON decoding failed for {file_name}: {e}")
            continue

    # phpMyAdmin export structure
    data_section = None
    for section in content:
        if section.get("type") == "table" and "data" in section:
            data_section = section["data"]
            break

    if not data_section:
        print(f"⚠️ No 'data' found in {file_name}")
        continue

    section_name = file_name
    count = 0

    for i, record in enumerate(data_section):
        emp_id = record.get("emp_id") or record.get("empid")
        if not emp_id:
            print(f"⚠️ No emp_id in record {i} of {file_name}")
            continue
        employee_data[emp_id][section_name].append(record)
        count += 1

    print(f"✅ Loaded {count} records from {file_name}")

# Flatten employee_master (optional)
for emp_id, sections in employee_data.items():
    if "employee_master" in sections and isinstance(sections["employee_master"], list):
        sections["employee_master"] = sections["employee_master"][0]

# Save final output
output_path = os.path.join(json_directory, "full_employee_json_data.json")
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(employee_data, f, indent=4)

print(f"\n💾 Final merged JSON saved to: {output_path}")

# Optional: show one sample record
if employee_data:
    first_emp_id = next(iter(employee_data))
    print(f"\n🔎 Sample record for emp_id = {first_emp_id}:")
    print(json.dumps(employee_data[first_emp_id], indent=2))
else:
    print("⚠️ No employee data found. Check JSON files again.")


# JSON TO MYSQL Migrator

In [ ]:
import json
import mysql.connector
from datetime import datetime
import os
import glob

# --- Database Configuration ---
# !!! REPLACE WITH YOUR ACTUAL DATABASE CREDENTIALS !!!
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': '',
    'database': 'user_master_db' # Make sure this matches your new DB name
}

# --- Database Reset Function ---
def reset_database_tables():
    """
    Truncates all relevant tables and resets their AUTO_INCREMENT counters to 1.
    """
    tables_to_reset = [
        'user_work_experience',
        'user_references',
        'user_languages',
        'user_it_details',
        'user_hr_details',
        'user_education',
        'user_certifications',
        'user_bank_details',
        'spouse_details',
        'parent_details',
        'users' 
    ]
    
    conn = None
    try:
        conn = get_db_connection()
        if not conn:
            print("Error: Could not connect to the database to perform reset.")
            return False
            
        cursor = conn.cursor()
        
        print("\n--- Resetting Database Tables ---")
        
        cursor.execute("SET FOREIGN_KEY_CHECKS = 0;")
        
        for table in tables_to_reset:
            try:
                print(f"Resetting table: {table}...")
                cursor.execute(f"TRUNCATE TABLE {table};")
                cursor.execute(f"ALTER TABLE {table} AUTO_INCREMENT = 1;")
            except mysql.connector.Error as err:
                if "Unknown table" in str(err):
                     print(f"  - Warning: Table '{table}' not found. Skipping.")
                else:
                    raise 
        
        conn.commit()
        print("Database tables have been reset successfully.")
        return True

    except mysql.connector.Error as err:
        print(f"Database error during reset: {err}")
        if conn: conn.rollback()
        return False
    finally:
        if conn and conn.is_connected():
            cursor.execute("SET FOREIGN_KEY_CHECKS = 1;")
            cursor.close()
            conn.close()

# --- Helper Functions ---
def get_db_connection():
    try:
        conn = mysql.connector.connect(**db_config)
        return conn
    except mysql.connector.Error as err:
        print(f"Error connecting to MySQL: {err}")
        return None

def format_date(date_str, input_formats=['%Y-%m-%d', '%d-%m-%Y', '%m/%d/%Y', '%d-%b-%Y %H:%M:%S']): 
    if not date_str: return None
    for fmt in input_formats:
        try:
            dt_obj = datetime.strptime(str(date_str), fmt)
            return dt_obj.strftime('%Y-%m-%d')
        except (ValueError, TypeError):
            continue
    return None

def to_boolean(value_str, true_val_primary="1", true_val_alternates=None):
    if value_str is None: return None 
    s_input = str(value_str).strip().lower()
    if not s_input: return None
    true_conditions = [str(true_val_primary).lower()] 
    if true_val_alternates:
        for alt in true_val_alternates:
            if alt is not None and isinstance(alt, str):
                true_conditions.append(alt.lower())
    return 1 if s_input in true_conditions else 0

def get_lookup_value(id_value, lookup_type, default_if_not_found=None):
    if not id_value: return default_if_not_found
    if lookup_type == 'country':
        if str(id_value) == "101": return "India"
    elif lookup_type == 'state':
        if str(id_value) == "7": return "Chhattisgarh" 
    return default_if_not_found if default_if_not_found is not None else str(id_value)

def sanitize_filename(filename):
    if not filename: return None
    sane_filename = "".join(c if c.isalnum() or c in ['.', '_', '-'] else '_' for c in str(filename))
    sane_filename = "_".join(filter(None, sane_filename.split('_')))
    sane_filename = ".".join(filter(None, sane_filename.split('.'))) 
    if len(sane_filename) > 200: 
        name, ext = os.path.splitext(sane_filename)
        sane_filename = name[:200-len(ext)] + ext
    return sane_filename

def find_original_filename_from_emp_docs(emp_documents, old_internal_emp_id_for_lookup, doc_type_in_emp_docs, related_old_id=None, old_table_name=None):
    if not emp_documents: return None
    for doc in emp_documents:
        if str(doc.get('empid')) == str(old_internal_emp_id_for_lookup) and doc.get('document_name') == doc_type_in_emp_docs:
            if related_old_id is not None and str(doc.get('field_id')) != str(related_old_id):
                continue
            if old_table_name is not None and doc.get('field_table_name') != old_table_name:
                continue
            original_filename = doc.get('document')
            if original_filename and str(original_filename).strip():
                return str(original_filename).strip() 
    return None

# --- Main Data Processing Function ---
def migrate_employee_data(all_data_for_single_employee): 
    conn = get_db_connection()
    if not conn: return False
    cursor = conn.cursor(dictionary=True) 
    new_user_id = None 

    data = all_data_for_single_employee.get('employee_master') 
    if not data or not isinstance(data, dict): 
        print(f"Error: employee_master data is missing or not a dictionary.")
        return False
        
    # Corrected keys are used to retrieve data from the JSON
    emp_contact_list = all_data_for_single_employee.get('emp_contact', [])
    emp_family_list = all_data_for_single_employee.get('emp_family', [])
    emp_education_list = all_data_for_single_employee.get('emp_education', [])
    emp_experience_list = all_data_for_single_employee.get('emp_experience', []) 
    emp_languages_list = all_data_for_single_employee.get('emp_language', []) # CORRECTED KEY
    emp_references_list = all_data_for_single_employee.get('emp_references', []) # CORRECTED KEY
    emp_documents_list = all_data_for_single_employee.get('emp_documents', [])
    emp_hobbies_list = all_data_for_single_employee.get('emp_hobbies', [])
    emp_curricular_list = all_data_for_single_employee.get('emp_curricular', [])
    emp_certification_list = all_data_for_single_employee.get('emp_certification', [])
    
    old_internal_emp_id = data.get('emp_id') 
    employee_id_ascent_val = data.get('User_id') 

    if not old_internal_emp_id or not str(old_internal_emp_id).strip():
        print(f"Error: 'emp_id' is missing or invalid in employee_master data.")
        return False

    print(f"\n--- Processing records for old_emp_id: {old_internal_emp_id} ---")

    try:
        contact_info = emp_contact_list[0] if emp_contact_list else {}

        pan_available_bool = 1 if data.get('pan_no') and str(data.get('pan_no')).strip() else 0
        aadhar_available_bool = to_boolean(data.get('aadhaar_card'), true_val_primary="Available")
        if aadhar_available_bool is None : aadhar_available_bool = 0 
        dl_available_bool = 1 if data.get('licence_no') and str(data.get('licence_no')).strip() else 0
        passport_available_bool = 1 if data.get('passport_no') and str(data.get('passport_no')).strip() else 0
        has_past_experience_bool = to_boolean(data.get('past_experience'), "1")
        has_pf_account_bool = 1 if (data.get('pf_account') and str(data.get('pf_account')).strip()) or \
                                     (data.get('uan_no') and str(data.get('uan_no')).strip()) else 0
        medical_disability_exists_bool = to_boolean(data.get('medical_disability_status'), true_val_primary="no", true_val_alternates=["yes"]) 
        if medical_disability_exists_bool == 0 and str(data.get('medical_disability_status')).strip().lower() == "no": medical_disability_exists_bool = 0
        elif medical_disability_exists_bool == 1 and str(data.get('medical_disability_status')).strip().lower() == "yes": medical_disability_exists_bool = 1
        else: medical_disability_exists_bool = 0 if medical_disability_exists_bool is None else medical_disability_exists_bool
        prev_employer_liability_exists_bool = to_boolean(data.get('any_libility'), true_val_primary="yes")
        worked_simplex_group_bool = to_boolean(data.get('applied_before'), true_val_primary="yes")
        agree_posted_anywhere_india_bool = to_boolean(data.get('agree_posted_anywhere'), true_val_primary="yes")
        declaration_agreed_bool = 1 
        hobbies_str = ", ".join([h.get('hobbie_name', '') for h in emp_hobbies_list if h.get('hobbie_name')]) if emp_hobbies_list else None
        extra_curricular_str = ", ".join([c.get('curricular', '') for c in emp_curricular_list if c.get('curricular')]) if emp_curricular_list else None
        profile_pic_filename = sanitize_filename(data.get('profile_pic'))
        profile_pic_path = f"uploads/profile/{profile_pic_filename}" if profile_pic_filename else None
        signature_filename = sanitize_filename(data.get('emp_signature'))
        signature_path = f"uploads/signature/{signature_filename}" if signature_filename else None
        pan_doc_filename = find_original_filename_from_emp_docs(emp_documents_list, old_internal_emp_id, 'pan', old_table_name='employee_master')
        pan_card_file_path = f"uploads/attachments/{old_internal_emp_id}/{sanitize_filename(pan_doc_filename)}" if pan_doc_filename and pan_available_bool else None
        aadhar_doc_filename = find_original_filename_from_emp_docs(emp_documents_list, old_internal_emp_id, 'aadhaar', old_table_name='employee_master')
        aadhar_card_file_path = f"uploads/attachments/{old_internal_emp_id}/{sanitize_filename(aadhar_doc_filename)}" if aadhar_doc_filename and aadhar_available_bool else None
        dl_doc_filename = find_original_filename_from_emp_docs(emp_documents_list, old_internal_emp_id, 'licence', old_table_name='employee_master')
        dl_file_path = f"uploads/attachments/{old_internal_emp_id}/{sanitize_filename(dl_doc_filename)}" if dl_doc_filename and dl_available_bool else None
        passport_doc_filename = find_original_filename_from_emp_docs(emp_documents_list, old_internal_emp_id, 'passport', old_table_name='employee_master')
        passport_file_path = f"uploads/attachments/{old_internal_emp_id}/{sanitize_filename(passport_doc_filename)}" if passport_doc_filename and passport_available_bool else None
        registration_date_formatted = format_date(data.get('registration_date'), input_formats=['%Y-%m-%d %H:%M:%S', '%d-%b-%Y %H:%M:%S'])
        registration_timestamp_val = registration_date_formatted + datetime.now().strftime(" %H:%M:%S") if registration_date_formatted else datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        sql_insert_user = """
            INSERT INTO users (
                name_as_for_document, salutation, first_name, middle_name, surname, nationality, gender, religion, category_type,
                date_of_birth, celebrated_date_of_birth, perm_birth_country, perm_birth_state, perm_birth_city_village,
                perm_address_line1, perm_address_line2, perm_address_line3, present_birth_country, present_birth_state,
                present_birth_city_village, present_address_line1, present_address_line2, present_address_line3,
                blood_group, weight_kg, height_cm, identification_marks, pan_available, pan_card_no, pan_card_file_path,
                aadhar_available, aadhar_number, aadhar_card_file_path, dl_available, dl_number, dl_file_path,
                dl_vehicle_type, dl_expiration_date, passport_available, passport_number, passport_file_path,
                passport_expiration_date, profile_picture_path, signature_path, marital_status, your_email_id,
                your_phone_number, emergency_contact_number, has_past_experience, has_pf_account, pf_account_established_code,
                pf_uan_no, pf_esi_no, extra_curricular_activities, hobbies, medical_disability_exists,
                medical_disability_details, prev_employer_liability_exists, prev_employer_liability_details,
                worked_simplex_group, agree_posted_anywhere_india, declaration_agreed, registration_timestamp 
            ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
        """
        user_values = (
            data.get('name_as_for'), data.get('empSalutation'), data.get('empFirstName','').strip(), data.get('empMiddleName') or None, data.get('empSurname') or None,
            data.get('nationality'), data.get('gender'), data.get('religion'), data.get('cast_category'),
            format_date(data.get('emp_dob')), format_date(data.get('emp_dobClb')),
            get_lookup_value(data.get('dob_country'), 'country'), get_lookup_value(data.get('dob_state'), 'state'), data.get('dob_city'),
            ( (data.get('permanent_hno') or '') + ' ' + (data.get('permanent_streetno') or '') + ' ' + (data.get('permanent_addr') or '')).strip() or None,
            None, data.get('permanent_pin'),
            get_lookup_value(data.get('present_country'), 'country'), get_lookup_value(data.get('present_state'), 'state'), data.get('present_city'),
            ( (data.get('present_hno') or '') + ' ' + (data.get('present_streetno') or '') + ' ' + (data.get('present_addr') or '')).strip() or None,
            None, data.get('present_pin'),
            data.get('blood_group', '').replace(' ', '') if data.get('blood_group') else None,
            float(data.get('emp_weight')) if data.get('emp_weight') and str(data.get('emp_weight')).replace('.','',1).isdigit() else None,
            float(data.get('emp_height')) if data.get('emp_height') and str(data.get('emp_height')).replace('.','',1).isdigit() else None,
            data.get('identification'),
            pan_available_bool, data.get('pan_no') if pan_available_bool else None, pan_card_file_path,
            aadhar_available_bool, data.get('aadhaar_no') if aadhar_available_bool else None, aadhar_card_file_path,
            dl_available_bool, data.get('licence_no') if dl_available_bool else None, dl_file_path, data.get('vehicle_type') if dl_available_bool else None, format_date(data.get('licence_expiry')) if dl_available_bool else None,
            passport_available_bool, data.get('passport_no') if passport_available_bool else None, passport_file_path, format_date(data.get('passport_expiry')) if passport_available_bool else None,
            profile_pic_path, signature_path,
            data.get('marital_status'),
            contact_info.get('Personal_Email', data.get('User_Name')), 
            contact_info.get('Personal_Mobile', contact_info.get('Personal_Phone')), 
            None, 
            has_past_experience_bool, has_pf_account_bool, data.get('pf_account') if has_pf_account_bool else None, 
            data.get('uan_no') if has_pf_account_bool else None, data.get('esi_no') if has_pf_account_bool else None,
            extra_curricular_str, hobbies_str,
            medical_disability_exists_bool, data.get('medical_disability_detail') if medical_disability_exists_bool else None,
            prev_employer_liability_exists_bool, data.get('libility_detail') if prev_employer_liability_exists_bool else None,
            worked_simplex_group_bool, agree_posted_anywhere_india_bool,
            declaration_agreed_bool, registration_timestamp_val
        )
        cursor.execute(sql_insert_user, user_values)
        new_user_id = cursor.lastrowid 
        print(f"-> Inserted base user with new_user_id: {new_user_id}")

        # --- II. spouse_details Table ---
        if data.get('marital_status') in ['Married', 'Registered Partnership'] :
            spouse_name_from_master = ( (data.get('fatherSpouseFirstName') or '') + ' ' + (data.get('fatherSpouseMiddleName') or '') + ' ' + (data.get('fatherSpouseSurname') or '') ).strip()
            spouse_salutation_from_master = data.get('fatherSpouseSalutation')
            spouse_data_to_insert = {'salutation': spouse_salutation_from_master, 'name': spouse_name_from_master, 'mobile_number': None, 'date_of_birth': None, 'aadhar_no': None, 'occupation': None, 'address': None, 'is_nominee_pf': 0, 'is_nominee_esic': 0, 'is_dependent': 0 }
            found_spouse_in_family = False
            for family_member in emp_family_list:
                if str(family_member.get('emp_id')) == str(old_internal_emp_id) and family_member.get('member_relation', '').lower() == 'spouse': 
                    spouse_data_to_insert['name'] = family_member.get('member_name', spouse_name_from_master)
                    spouse_data_to_insert['salutation'] = family_member.get('member_salutation', spouse_salutation_from_master)
                    spouse_data_to_insert['mobile_number'] = family_member.get('member_contact')
                    spouse_data_to_insert['date_of_birth'] = format_date(family_member.get('member_dob'))
                    spouse_data_to_insert['aadhar_no'] = family_member.get('member_aadhaar')
                    spouse_data_to_insert['occupation'] = family_member.get('member_occupation')
                    spouse_data_to_insert['address'] = family_member.get('member_address')
                    spouse_data_to_insert['is_nominee_pf'] = to_boolean(family_member.get('pf_nominee'))
                    spouse_data_to_insert['is_nominee_esic'] = to_boolean(family_member.get('esic_nominee'))
                    spouse_data_to_insert['is_dependent'] = to_boolean(family_member.get('dependent'))
                    found_spouse_in_family = True
                    break
            if spouse_data_to_insert['name']: 
                sql_insert_spouse = "INSERT INTO spouse_details (user_id, salutation, name, mobile_number, date_of_birth, aadhar_no, occupation, address, is_nominee_pf, is_nominee_esic, is_dependent) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
                spouse_values = (new_user_id, spouse_data_to_insert['salutation'], spouse_data_to_insert['name'], spouse_data_to_insert['mobile_number'], spouse_data_to_insert['date_of_birth'], spouse_data_to_insert['aadhar_no'], spouse_data_to_insert['occupation'], spouse_data_to_insert['address'], spouse_data_to_insert['is_nominee_pf'], spouse_data_to_insert['is_nominee_esic'], spouse_data_to_insert['is_dependent'])
                cursor.execute(sql_insert_spouse, spouse_values)
                print(f"-> Inserted spouse details.")

        # --- III. parent_details Table ---
        for family_member in emp_family_list:
            if str(family_member.get('emp_id')) == str(old_internal_emp_id) and family_member.get('member_relation', '').lower() in ['father', 'mother']:
                parent_type = family_member.get('member_relation', '').capitalize()
                parent_name = family_member.get('member_name') 
                parent_salutation = family_member.get('member_salutation')
                parent_mobile = family_member.get('member_contact')
                parent_dob = format_date(family_member.get('member_dob'))
                parent_aadhar = family_member.get('member_aadhaar')
                parent_aadhar_doc_filename = find_original_filename_from_emp_docs(emp_documents_list, old_internal_emp_id, 'family', family_member.get('member_id'), 'emp_family')
                parent_aadhar_file = f"uploads/attachments/{old_internal_emp_id}/{sanitize_filename(parent_aadhar_doc_filename)}" if parent_aadhar_doc_filename else None
                parent_occupation = family_member.get('member_occupation')
                parent_address = family_member.get('member_address')
                parent_nom_pf = to_boolean(family_member.get('pf_nominee'))
                parent_nom_esic = to_boolean(family_member.get('esic_nominee'))
                parent_dependent = to_boolean(family_member.get('dependent'))
                sql_insert_parent = "INSERT INTO parent_details (user_id, parent_type, salutation, name, mobile_number, date_of_birth, aadhar_no, aadhar_file_path, occupation, address, is_nominee_pf, is_nominee_esic, is_dependent) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
                parent_values = (new_user_id, parent_type, parent_salutation, parent_name, parent_mobile, parent_dob, parent_aadhar, parent_aadhar_file, parent_occupation, parent_address, parent_nom_pf, parent_nom_esic, parent_dependent)
                cursor.execute(sql_insert_parent, parent_values)
        print(f"-> Processed parent details from emp_family.")

        # --- IV. user_education Table ---
        for edu_record in emp_education_list:
            if str(edu_record.get('emp_id')) == str(old_internal_emp_id):
                edu_doc_filename = find_original_filename_from_emp_docs(emp_documents_list, old_internal_emp_id, 'education', edu_record.get('eid'), 'emp_education')
                edu_doc_path = f"uploads/attachments/{old_internal_emp_id}/{sanitize_filename(edu_doc_filename)}" if edu_doc_filename else None
                sql_edu = "INSERT INTO user_education (user_id, qualification, board_university, subject, enrollment_year, passing_year, percentage_grade, document_path) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
                edu_values = (new_user_id, edu_record.get('education_type'), edu_record.get('university'), edu_record.get('major_subject'), edu_record.get('from_year'), edu_record.get('to_year'), edu_record.get('gradepoint'), edu_doc_path)
                cursor.execute(sql_edu, edu_values)
        print(f"-> Processed {len(emp_education_list)} education records.")

        # --- V. user_certifications Table --- 
        for cert_record in emp_certification_list:
            if str(cert_record.get('emp_id')) == str(old_internal_emp_id):
                cert_doc_filename = find_original_filename_from_emp_docs(emp_documents_list, old_internal_emp_id, 'certification', cert_record.get('cid'), 'emp_certification') 
                cert_doc_path = f"uploads/attachments/{old_internal_emp_id}/{sanitize_filename(cert_doc_filename)}" if cert_doc_filename else None
                sql_cert = "INSERT INTO user_certifications (user_id, certificate_name, issued_on, valid_upto, certificate_authority, document_path) VALUES (%s, %s, %s, %s, %s, %s)"
                cert_values = (new_user_id, cert_record.get('certificate_name'), format_date(cert_record.get('issued_on')), format_date(cert_record.get('valid_upto')), cert_record.get('certificate_authority'), cert_doc_path)
                cursor.execute(sql_cert, cert_values)
        print(f"-> Processed {len(emp_certification_list)} certification records.")

        # --- VI. user_work_experience Table ---
        for exp_record in emp_experience_list:
            if str(exp_record.get('emp_id')) == str(old_internal_emp_id):
                exp_letter_filename = find_original_filename_from_emp_docs(emp_documents_list, old_internal_emp_id, 'experience_letter', exp_record.get('exp_id'), 'emp_experience') # CORRECTED TYPO
                exp_letter_path = f"uploads/attachments/{old_internal_emp_id}/{sanitize_filename(exp_letter_filename)}" if exp_letter_filename else None
                sql_exp = "INSERT INTO user_work_experience (user_id, company_name, designation, reason_for_leaving, salary_per_annum, roles_responsibility, competency, from_date, to_date, employer_contact_no, experience_letter_path) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
                exp_values = (new_user_id, exp_record.get('company_name'), exp_record.get('designation'), exp_record.get('leaving_resion'), exp_record.get('salary'), exp_record.get('roles_responsibility'), exp_record.get('competency'), format_date(exp_record.get('from_year')), format_date(exp_record.get('to_year')), exp_record.get('employer_contact'), exp_letter_path)
                cursor.execute(sql_exp, exp_values)
        print(f"-> Processed {len(emp_experience_list)} work experience records.")

        # --- VII. user_languages Table ---
        for lang_record in emp_languages_list:
            if str(lang_record.get('emp_id')) == str(old_internal_emp_id):
                sql_lang = "INSERT INTO user_languages (user_id, language_name, can_speak, can_read, can_write, can_understand) VALUES (%s, %s, %s, %s, %s, %s)"
                lang_values = (new_user_id, lang_record.get('lang_name'), to_boolean(lang_record.get('lang_speak')), to_boolean(lang_record.get('lang_read')), to_boolean(lang_record.get('lang_write')), to_boolean(lang_record.get('lang_understand')))
                cursor.execute(sql_lang, lang_values)
        print(f"-> Processed {len(emp_languages_list)} language records.")

        # --- VIII. user_references Table ---
        for ref_record in emp_references_list:
            if str(ref_record.get('emp_id')) == str(old_internal_emp_id):
                sql_ref = "INSERT INTO user_references (user_id, reference_name, address, designation_position, relation, contact_no) VALUES (%s, %s, %s, %s, %s, %s)"
                ref_values = (new_user_id, ref_record.get('ref_name'), ref_record.get('ref_address'), ref_record.get('ref_designation'), ref_record.get('ref_relation'), ref_record.get('ref_contact'))
                cursor.execute(sql_ref, ref_values)
        print(f"-> Processed {len(emp_references_list)} reference records.")
        
        # --- IX. user_bank_details Table ---
        if data.get('bank_name'): 
            bank_passbook_filename = find_original_filename_from_emp_docs(emp_documents_list, old_internal_emp_id, 'passbook', old_table_name='employee_master') 
            bank_passbook_path = f"uploads/attachments/{old_internal_emp_id}/{sanitize_filename(bank_passbook_filename)}" if bank_passbook_filename else None
            sql_insert_bank = "INSERT INTO user_bank_details (user_id, bank_name, account_number, ifsc_code, micr_code, bank_address, passbook_document_path) VALUES (%s, %s, %s, %s, %s, %s, %s)"
            bank_values = (new_user_id, data.get('bank_name'), data.get('bank_account'), data.get('ifsc_no'), data.get('micr_code'), data.get('bank_addr'), bank_passbook_path)
            cursor.execute(sql_insert_bank, bank_values)
            print("-> Inserted bank details.")

        # --- X. user_hr_details Table ---
        sql_insert_hr = "INSERT INTO user_hr_details (user_id, unit, department, designation, date_of_joining, category, grade, status, leave_group, shift_schedule, reporting_incharge, department_head, attendance_policy, employee_id_ascent, employee_role, payroll_code, vaccination_code, employee_portal_status) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        hr_values = (
            new_user_id, data.get('unit'), data.get('department'), data.get('postName'), 
            format_date(data.get('joiningDate')), data.get('Category'), data.get('grade'),     
            data.get('ascent_status'), data.get('leave_group'), data.get('shift_schedule'), data.get('Reporting_group'),
            data.get('emp_head_id'), data.get('attendance_policy'), employee_id_ascent_val, 
            'USER', data.get('payroll_code'), data.get('vacci_code'),
            data.get('emp_status')
        )
        cursor.execute(sql_insert_hr, hr_values)
        print("-> Inserted HR details.")
        
        # --- XI. user_it_details Table ---
        it_official_phone = contact_info.get('Official_Mobile', contact_info.get('Official_Phone'))
        it_official_email = contact_info.get('Official_Email')
        it_intercom_number = contact_info.get('Official_Extn')
        if it_official_phone or it_official_email or it_intercom_number:
            sql_insert_it = "INSERT INTO user_it_details (user_id, official_phone_number, official_email, intercom_number) VALUES (%s, %s, %s, %s)"
            it_values = (new_user_id, it_official_phone, it_official_email, it_intercom_number)
            cursor.execute(sql_insert_it, it_values)
            print("-> Inserted IT details.")
        
        conn.commit()
        print(f"-> Successfully migrated data for old internal emp_id: {old_internal_emp_id}")
        return True

    except mysql.connector.Error as err:
        print(f"Database error during migration for old internal emp_id {old_internal_emp_id}: {err}")
        if conn.is_connected(): conn.rollback()
        return False
    except Exception as e:
        print(f"An unexpected error occurred for old internal emp_id {old_internal_emp_id}: {e}")
        if conn.is_connected(): conn.rollback()
        return False
    finally:
        if conn and conn.is_connected():
            cursor.close()
            conn.close()

# --- Main Execution Block ---
if __name__ == "__main__":
    if not reset_database_tables():
        print("\nHalting migration due to database reset failure.")
        exit()

    unified_json_file_path = "json/full_employee_json_data.json" 
    
    try:
        with open(unified_json_file_path, 'r', encoding='utf-8') as f:
            all_employees_data_from_file = json.load(f)
        print(f"Successfully loaded unified JSON file: {unified_json_file_path}")
    except FileNotFoundError:
        print(f"Error: Unified JSON file not found - {unified_json_file_path}.")
        exit()
    except json.JSONDecodeError:
        print(f"Error: Could not decode JSON from file - {unified_json_file_path}")
        exit()

    if not isinstance(all_employees_data_from_file, dict):
        print(f"Error: Unified JSON file is not a dictionary.")
        exit()

    total_employees = len(all_employees_data_from_file)
    success_count = 0
    failure_count = 0
    failed_entries = [] # List to store the IDs of failed entries

    print(f"\nStarting migration for {total_employees} employee(s)...")

    for old_emp_id_key, employee_data_struct in all_employees_data_from_file.items():
        if migrate_employee_data(employee_data_struct):
            success_count += 1
        else:
            failure_count += 1
            failed_entries.append(old_emp_id_key) # Add the failed ID to the list
        print("-" * 50)

    print("\n--- Migration Summary ---")
    print(f"Total entries in JSON: {total_employees}")
    print(f"Successfully migrated: {success_count}")
    print(f"Failed or skipped: {failure_count}")

    if failed_entries:
        print("\n--- Failed or Skipped emp_id's ---")
        # Print the list of failed IDs, 10 per line for readability
        for i in range(0, len(failed_entries), 10):
             print(", ".join(failed_entries[i:i+10]))

